In [3]:
import pandas as pd
from sqlalchemy import create_engine
import yaml

In [4]:
with open('C:/Users/WILSON/OneDrive/Escritorio/Septimo Semestre/CIENCIA DE DATOS/REPO_MONITOR/CS_etl_py/config.yml', 'r') as f:
    config = yaml.safe_load(f)
    config_co = config['CO_SA']
    
url_co = (f"{config_co['drivername']}://{config_co['user']}:{config_co['password']}@{config_co['host']}:{config_co['port']}/{config_co['dbname']}")

co_sa = create_engine(url_co)


In [6]:
def load_table(schema, table):
    query = f"SELECT * FROM {schema}.{table}"
    return pd.read_sql(query, co_sa)

In [7]:
salesorderheader = load_table("sales", "salesorderheader")
salesorderdetail = load_table("sales", "salesorderdetail")
customer = load_table("sales", "customer")
personcreditcard = load_table("sales", "personcreditcard")
salesperson = load_table("sales", "salesperson")
store = load_table("sales", "store")
specialoffer = load_table("sales", "specialoffer")
specialofferproduct = load_table("sales", "specialofferproduct")
salesterritory = load_table("sales", "salesterritory")
currency = load_table("sales", "currency")
currencyrate = load_table("sales", "currencyrate")

In [8]:
person = load_table("person", "person")
address = load_table("person", "address")
businessentity = load_table("person", "businessentity")
businessentityaddress = load_table("person", "businessentityaddress")
stateprovince = load_table("person", "stateprovince")
countryregion = load_table("person", "countryregion")


In [9]:
product = load_table("production", "product")
subcategory = load_table("production", "productsubcategory")
category = load_table("production", "productcategory")


In [10]:
employee = load_table("humanresources", "employee")
emp_history = load_table("humanresources", "employeedepartmenthistory")
department = load_table("humanresources", "department")


In [11]:
#DIMENCION DATOS
def build_dimdate(df, colname):
    date_df = pd.DataFrame()
    date_df["date"] = pd.to_datetime(df[colname].dropna().unique())

    date_df["datekey"] = date_df["date"].dt.strftime("%Y%m%d").astype(int)
    date_df["year"] = date_df["date"].dt.year
    date_df["month"] = date_df["date"].dt.month
    date_df["day"] = date_df["date"].dt.day
    date_df["quarter"] = date_df["date"].dt.quarter
    date_df["is_weekend"] = (date_df["date"].dt.weekday >= 5).astype(int)

    return date_df

dimdate_order = build_dimdate(salesorderheader, "orderdate")
dimdate_due = build_dimdate(salesorderheader, "duedate")
dimdate_ship = build_dimdate(salesorderheader, "shipdate")


In [12]:
currencyrate = currencyrate.rename(columns={'tocurrencycode': 'currencycode'})

In [13]:
#DIMENSION CURRENCY
dimcurrency = currency.merge(currencyrate, on="currencycode", how="left")

In [93]:
customer = customer.rename(columns={'personid': 'businessentityid'})
customer

,customerid,businessentityid,storeid,territoryid,accountnumber,rowguid,modifieddate
0,1,NaN,934.0,1,AW00000001,3f5ae95e-b87d-4aed-95b4-c3797afcb74f,2014-09-12 11:15:07.263000+00:00
1,2,NaN,1028.0,1,AW00000002,e552f657-a9af-4a7d-a645-c429d6e02491,2014-09-12 11:15:07.263000+00:00
2,3,NaN,642.0,4,AW00000003,130774b1-db21-4ef3-98c8-c104bcd6ed6d,2014-09-12 11:15:07.263000+00:00
3,4,NaN,932.0,4,AW00000004,ff862851-1daa-4044-be7c-3e85583c054d,2014-09-12 11:15:07.263000+00:00
4,5,NaN,1026.0,4,AW00000005,83905bdc-6f5e-4f71-b162-c98da069f38a,2014-09-12 11:15:07.263000+00:00
...,...,...,...,...,...,...,...
19815,30114,1985.0,1986.0,7,AW00030114,97154f3d-28f1-4b15-ae03-9518b781ece3,2014-09-12 11:15:07.263000+00:00
19816,30115,1987.0,1988.0,6,AW00030115,e4cf8fd5-30a4-4b8e-8fd8-47032e255778,2014-09-12 11:15:07.263000+00:00
19817,30116,1989.0,1990.0,4,AW00030116,ec409609-d25d-41b8-9d15-a1aa6e89fc77,2014-09-12 11:15:07.263000+00:00
19818,30117,1991.0,1992.0,4,AW00030117,6f08e2fb-1cd3-4f6e-a2e6-385669598b19,2014-09-12 11:15:07.263000+00:00


In [94]:
customer = customer.drop(columns=['rowguid', 'modifieddate','accountnumber'])
#customer = customer.drop(columns=['rowguid'])
customer

,customerid,businessentityid,storeid,territoryid
0,1,NaN,934.0,1
1,2,NaN,1028.0,1
2,3,NaN,642.0,4
3,4,NaN,932.0,4
4,5,NaN,1026.0,4
...,...,...,...,...
19815,30114,1985.0,1986.0,7
19816,30115,1987.0,1988.0,6
19817,30116,1989.0,1990.0,4
19818,30117,1991.0,1992.0,4


In [95]:
internet_customers = customer[customer["storeid"].isna()]
cust = (
    internet_customers
    .merge(person, on="businessentityid", how="left")
)

In [96]:
cust

,customerid,businessentityid,storeid,territoryid,persontype,namestyle,title,firstname,middlename,lastname,suffix,emailpromotion,additionalcontactinfo,demographics,rowguid,modifieddate
0,11000,13531.0,NaN,9,IN,False,None,Jon,V,Yang,None,1,None,"<IndividualSurvey xmlns=""http://schemas.micros...",66416a79-00c0-4dee-bc3e-662a3d8f6424,2011-06-21 00:00:00+00:00
1,11001,5454.0,NaN,9,IN,False,None,Eugene,L,Huang,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",82482f35-0f48-44e1-8a73-c11518b5a979,2011-06-17 00:00:00+00:00
2,11002,11269.0,NaN,9,IN,False,None,Ruben,None,Torres,None,2,None,"<IndividualSurvey xmlns=""http://schemas.micros...",ceb6b318-5d42-4e4f-9646-aa406758000e,2011-06-09 00:00:00+00:00
3,11003,11358.0,NaN,9,IN,False,None,Christy,None,Zhu,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",334c9ee5-03d3-4774-a886-5ec7ba93f034,2011-05-31 00:00:00+00:00
4,11004,11901.0,NaN,9,IN,False,None,Elizabeth,None,Johnson,None,1,None,"<IndividualSurvey xmlns=""http://schemas.micros...",8609c9c5-f804-4de8-bb5b-9344971bd670,2011-06-25 00:00:00+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18479,29479,4191.0,NaN,7,IN,False,None,Tommy,L,Tang,None,2,None,"<IndividualSurvey xmlns=""http://schemas.micros...",0acfc8d3-5415-4b2b-b5d3-214a21cf32e5,2013-02-04 00:00:00+00:00
18480,29480,4472.0,NaN,10,IN,False,None,Nina,W,Raji,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",3f4e9c34-35da-4483-b92d-e3bf98990a03,2013-12-17 00:00:00+00:00
18481,29481,8168.0,NaN,8,IN,False,None,Ivan,None,Suri,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",73cbf939-43c4-47ac-a025-bc3e7e3980df,2012-01-13 00:00:00+00:00
18482,29482,12570.0,NaN,7,IN,False,None,Clayton,None,Zhang,None,1,None,"<IndividualSurvey xmlns=""http://schemas.micros...",18eae52a-121b-40b9-895e-64150fa677f1,2013-02-18 00:00:00+00:00


In [142]:
#customer = customer[['customerid', 'businessentityid']]
#person = person[['businessentityid', 'firstname', 'lastname']]
# Customer → Person
#cust = customer.merge(person, on='businessentityid', how='left')
cust


,customerid,businessentityid,storeid,territoryid,persontype,namestyle,title,firstname,middlename,lastname,suffix,emailpromotion,additionalcontactinfo,demographics,rowguid,modifieddate
0,11000,13531.0,NaN,9,IN,False,None,Jon,V,Yang,None,1,None,"<IndividualSurvey xmlns=""http://schemas.micros...",66416a79-00c0-4dee-bc3e-662a3d8f6424,2011-06-21 00:00:00+00:00
1,11001,5454.0,NaN,9,IN,False,None,Eugene,L,Huang,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",82482f35-0f48-44e1-8a73-c11518b5a979,2011-06-17 00:00:00+00:00
2,11002,11269.0,NaN,9,IN,False,None,Ruben,None,Torres,None,2,None,"<IndividualSurvey xmlns=""http://schemas.micros...",ceb6b318-5d42-4e4f-9646-aa406758000e,2011-06-09 00:00:00+00:00
3,11003,11358.0,NaN,9,IN,False,None,Christy,None,Zhu,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",334c9ee5-03d3-4774-a886-5ec7ba93f034,2011-05-31 00:00:00+00:00
4,11004,11901.0,NaN,9,IN,False,None,Elizabeth,None,Johnson,None,1,None,"<IndividualSurvey xmlns=""http://schemas.micros...",8609c9c5-f804-4de8-bb5b-9344971bd670,2011-06-25 00:00:00+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18479,29479,4191.0,NaN,7,IN,False,None,Tommy,L,Tang,None,2,None,"<IndividualSurvey xmlns=""http://schemas.micros...",0acfc8d3-5415-4b2b-b5d3-214a21cf32e5,2013-02-04 00:00:00+00:00
18480,29480,4472.0,NaN,10,IN,False,None,Nina,W,Raji,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",3f4e9c34-35da-4483-b92d-e3bf98990a03,2013-12-17 00:00:00+00:00
18481,29481,8168.0,NaN,8,IN,False,None,Ivan,None,Suri,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",73cbf939-43c4-47ac-a025-bc3e7e3980df,2012-01-13 00:00:00+00:00
18482,29482,12570.0,NaN,7,IN,False,None,Clayton,None,Zhang,None,1,None,"<IndividualSurvey xmlns=""http://schemas.micros...",18eae52a-121b-40b9-895e-64150fa677f1,2013-02-18 00:00:00+00:00


In [97]:
cust = cust.rename(columns={"territoryid": "territoryidCustomer"})


In [98]:
cust = cust[['customerid','businessentityid','territoryidCustomer','firstname','lastname']]
cust

,customerid,businessentityid,territoryidCustomer,firstname,lastname
0,11000,13531.0,9,Jon,Yang
1,11001,5454.0,9,Eugene,Huang
2,11002,11269.0,9,Ruben,Torres
3,11003,11358.0,9,Christy,Zhu
4,11004,11901.0,9,Elizabeth,Johnson
...,...,...,...,...,...
18479,29479,4191.0,7,Tommy,Tang
18480,29480,4472.0,10,Nina,Raji
18481,29481,8168.0,8,Ivan,Suri
18482,29482,12570.0,7,Clayton,Zhang


In [99]:
businessentityaddress = businessentityaddress[['businessentityid', 'addressid']]
address = address[['addressid','addressline1', 'stateprovinceid', 'city']]
stateprovince = stateprovince[['stateprovinceid','countryregioncode', 'territoryid']]

In [100]:
# Address
cust = cust.merge(businessentityaddress, on="businessentityid", how="left")


In [101]:
#cust = cust.drop(columns=["addressid_x"])
#cust = cust.rename(columns={"addressid_y": "addressid"})
cust = cust[['customerid','businessentityid','firstname','lastname','addressid']]
cust

,customerid,businessentityid,firstname,lastname,addressid
0,11000,13531.0,Jon,Yang,22601
1,11001,5454.0,Eugene,Huang,14489
2,11002,11269.0,Ruben,Torres,20336
3,11003,11358.0,Christy,Zhu,20425
4,11004,11901.0,Elizabeth,Johnson,20971
...,...,...,...,...,...
18503,29479,4191.0,Tommy,Tang,13226
18504,29480,4472.0,Nina,Raji,13507
18505,29481,8168.0,Ivan,Suri,17218
18506,29482,12570.0,Clayton,Zhang,21640


In [102]:
cust = cust.merge(address, on="addressid", how="left")
cust


,customerid,businessentityid,firstname,lastname,addressid,addressline1,stateprovinceid,city
0,11000,13531.0,Jon,Yang,22601,3761 N. 14th St,64,Rockhampton
1,11001,5454.0,Eugene,Huang,14489,2243 W St.,77,Seaford
2,11002,11269.0,Ruben,Torres,20336,5844 Linden Land,71,Hobart
3,11003,11358.0,Christy,Zhu,20425,1825 Village Pl.,50,North Ryde
4,11004,11901.0,Elizabeth,Johnson,20971,7553 Harness Circle,50,Wollongong
...,...,...,...,...,...,...,...,...
18503,29479,4191.0,Tommy,Tang,13226,"111, rue Maillard",164,Versailles
18504,29480,4472.0,Nina,Raji,13507,9 Katherine Drive,14,London
18505,29481,8168.0,Ivan,Suri,17218,Knaackstr 4,8,Hof
18506,29482,12570.0,Clayton,Zhang,21640,"1080, quai de Grenelle",103,Saint Ouen


In [103]:
cust = cust.merge(stateprovince, on="stateprovinceid", how="left")
cust

,customerid,businessentityid,firstname,lastname,addressid,addressline1,stateprovinceid,city,countryregioncode,territoryid
0,11000,13531.0,Jon,Yang,22601,3761 N. 14th St,64,Rockhampton,AU,9
1,11001,5454.0,Eugene,Huang,14489,2243 W St.,77,Seaford,AU,9
2,11002,11269.0,Ruben,Torres,20336,5844 Linden Land,71,Hobart,AU,9
3,11003,11358.0,Christy,Zhu,20425,1825 Village Pl.,50,North Ryde,AU,9
4,11004,11901.0,Elizabeth,Johnson,20971,7553 Harness Circle,50,Wollongong,AU,9
...,...,...,...,...,...,...,...,...,...,...
18503,29479,4191.0,Tommy,Tang,13226,"111, rue Maillard",164,Versailles,FR,7
18504,29480,4472.0,Nina,Raji,13507,9 Katherine Drive,14,London,GB,10
18505,29481,8168.0,Ivan,Suri,17218,Knaackstr 4,8,Hof,DE,8
18506,29482,12570.0,Clayton,Zhang,21640,"1080, quai de Grenelle",103,Saint Ouen,FR,7


In [104]:
#cust = cust.drop(columns=["countryregioncode_x"])
#cust = cust.rename(columns={"countryregioncode_y": "countryregioncode"})
#cust = cust.drop(columns=["territoryid_x"])
#cust = cust.rename(columns={"territoryid_y": "territoryid"})

#DIMENCION CUSTOMER SOLO VENTAS POR INTERNET
dimcustomer = cust[[
    "customerid",
    "firstname",
    "lastname",
    "addressline1",
    "city",
    "stateprovinceid",
    "countryregioncode",
    "territoryid"
]].drop_duplicates()

cust

,customerid,businessentityid,firstname,lastname,addressid,addressline1,stateprovinceid,city,countryregioncode,territoryid
0,11000,13531.0,Jon,Yang,22601,3761 N. 14th St,64,Rockhampton,AU,9
1,11001,5454.0,Eugene,Huang,14489,2243 W St.,77,Seaford,AU,9
2,11002,11269.0,Ruben,Torres,20336,5844 Linden Land,71,Hobart,AU,9
3,11003,11358.0,Christy,Zhu,20425,1825 Village Pl.,50,North Ryde,AU,9
4,11004,11901.0,Elizabeth,Johnson,20971,7553 Harness Circle,50,Wollongong,AU,9
...,...,...,...,...,...,...,...,...,...,...
18503,29479,4191.0,Tommy,Tang,13226,"111, rue Maillard",164,Versailles,FR,7
18504,29480,4472.0,Nina,Raji,13507,9 Katherine Drive,14,London,GB,10
18505,29481,8168.0,Ivan,Suri,17218,Knaackstr 4,8,Hof,DE,8
18506,29482,12570.0,Clayton,Zhang,21640,"1080, quai de Grenelle",103,Saint Ouen,FR,7


In [105]:

#---------DIMENSION REVENDEDORES

reseller = customer[customer["storeid"].notnull()]
storeRevendedoreid = store
storeRevendedoreid = storeRevendedoreid.rename(columns={'businessentityid':'storeid'})
reseller = reseller.merge(storeRevendedoreid, on="storeid", how="left")


In [106]:
dimreseller = reseller[[
    "customerid",
    "storeid",
    "name",
    "businessentityid"
]].drop_duplicates()
dimreseller

,customerid,storeid,name,businessentityid
0,1,934.0,A Bike Store,NaN
1,2,1028.0,Progressive Sports,NaN
2,3,642.0,Advanced Bike Components,NaN
3,4,932.0,Modular Cycle Systems,NaN
4,5,1026.0,Metropolitan Sports Supply,NaN
...,...,...,...,...
1331,30114,1986.0,Recreation Toy Store,1985.0
1332,30115,1988.0,Retreat Inn,1987.0
1333,30116,1990.0,Technical Parts Manufacturing,1989.0
1334,30117,1992.0,Totes & Baskets Company,1991.0


In [44]:
#DIMENSION EMPLEADO


In [70]:

employeeMerge = employee.drop(columns={'rowguid','modifieddate'})
dimemployee = employeeMerge.merge(emp_history, on="businessentityid", how="left")
dimemployee = dimemployee.drop(columns={'modifieddate'})
dimemployee

,businessentityid,nationalidnumber,loginid,organizationnode,organizationlevel,jobtitle,birthdate,maritalstatus,gender,hiredate,salariedflag,vacationhours,sickleavehours,currentflag,departmentid,shiftid,startdate,enddate
0,1,295847284,adventure-works\ken0,None,NaN,Chief Executive Officer,1969-01-29,S,M,2009-01-14,True,99,69,True,16,1,2009-01-14,None
1,2,245797967,adventure-works\terri0,[b'X'],1.0,Vice President of Engineering,1971-08-01,S,F,2008-01-31,True,1,20,True,1,1,2008-01-31,None
2,3,509647174,adventure-works\roberto0,"[b'Z', b'\xc0']",2.0,Engineering Manager,1974-11-12,M,M,2007-11-11,True,2,21,True,1,1,2007-11-11,None
3,4,112457891,adventure-works\rob0,"[b'Z', b'\xd6']",3.0,Senior Tool Designer,1974-12-23,S,M,2007-12-05,False,48,80,True,1,1,2007-12-05,2010-05-30
4,4,112457891,adventure-works\rob0,"[b'Z', b'\xd6']",3.0,Senior Tool Designer,1974-12-23,S,M,2007-12-05,False,48,80,True,2,1,2010-05-31,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
291,286,758596752,adventure-works\lynn0,"[b'\x95', b'\xab']",3.0,Sales Representative,1977-02-14,S,F,2013-05-30,True,36,38,True,3,1,2013-05-30,None
292,287,982310417,adventure-works\amy0,"[b'\x95', b'\xe0']",2.0,European Sales Manager,1957-09-20,M,F,2012-04-16,True,21,30,True,3,1,2012-04-16,None
293,288,954276278,adventure-works\rachel0,"[b'\x95', b'\xeb']",3.0,Sales Representative,1975-07-09,S,F,2013-05-30,True,35,37,True,3,1,2013-05-30,None
294,289,668991357,adventure-works\jae0,"[b'\x95', b'\xed']",3.0,Sales Representative,1968-03-17,M,F,2012-05-30,True,37,38,True,3,1,2012-05-30,None


In [71]:
#departmentMerge = department.drop(columns={'modifieddate'})
#dimemployee.merge(departmentMerge, on="departmentid", how="left")
dimemployee = dimemployee[['businessentityid','jobtitle', 'loginid', 'departmentid' ]]
dimemployee


,businessentityid,jobtitle,loginid,departmentid
0,1,Chief Executive Officer,adventure-works\ken0,16
1,2,Vice President of Engineering,adventure-works\terri0,1
2,3,Engineering Manager,adventure-works\roberto0,1
3,4,Senior Tool Designer,adventure-works\rob0,1
4,4,Senior Tool Designer,adventure-works\rob0,2
...,...,...,...,...
291,286,Sales Representative,adventure-works\lynn0,3
292,287,European Sales Manager,adventure-works\amy0,3
293,288,Sales Representative,adventure-works\rachel0,3
294,289,Sales Representative,adventure-works\jae0,3


In [72]:
department

,departmentid,name,groupname,modifieddate
0,1,Engineering,Research and Development,2008-04-30 00:00:00+00:00
1,2,Tool Design,Research and Development,2008-04-30 00:00:00+00:00
2,3,Sales,Sales and Marketing,2008-04-30 00:00:00+00:00
3,4,Marketing,Sales and Marketing,2008-04-30 00:00:00+00:00
4,5,Purchasing,Inventory Management,2008-04-30 00:00:00+00:00
5,6,Research and Development,Research and Development,2008-04-30 00:00:00+00:00
6,7,Production,Manufacturing,2008-04-30 00:00:00+00:00
7,8,Production Control,Manufacturing,2008-04-30 00:00:00+00:00
8,9,Human Resources,Executive General and Administration,2008-04-30 00:00:00+00:00
9,10,Finance,Executive General and Administration,2008-04-30 00:00:00+00:00


In [73]:
departmentMerge = department
departmentMerge = departmentMerge[['departmentid', 'name', 'groupname']]
dimemployee = dimemployee.merge(departmentMerge, on="departmentid", how="left")
dimemployee

,businessentityid,jobtitle,loginid,departmentid,name,groupname
0,1,Chief Executive Officer,adventure-works\ken0,16,Executive,Executive General and Administration
1,2,Vice President of Engineering,adventure-works\terri0,1,Engineering,Research and Development
2,3,Engineering Manager,adventure-works\roberto0,1,Engineering,Research and Development
3,4,Senior Tool Designer,adventure-works\rob0,1,Engineering,Research and Development
4,4,Senior Tool Designer,adventure-works\rob0,2,Tool Design,Research and Development
...,...,...,...,...,...,...
291,286,Sales Representative,adventure-works\lynn0,3,Sales,Sales and Marketing
292,287,European Sales Manager,adventure-works\amy0,3,Sales,Sales and Marketing
293,288,Sales Representative,adventure-works\rachel0,3,Sales,Sales and Marketing
294,289,Sales Representative,adventure-works\jae0,3,Sales,Sales and Marketing


In [74]:
dimemployee

,businessentityid,jobtitle,loginid,departmentid,name,groupname
0,1,Chief Executive Officer,adventure-works\ken0,16,Executive,Executive General and Administration
1,2,Vice President of Engineering,adventure-works\terri0,1,Engineering,Research and Development
2,3,Engineering Manager,adventure-works\roberto0,1,Engineering,Research and Development
3,4,Senior Tool Designer,adventure-works\rob0,1,Engineering,Research and Development
4,4,Senior Tool Designer,adventure-works\rob0,2,Tool Design,Research and Development
...,...,...,...,...,...,...
291,286,Sales Representative,adventure-works\lynn0,3,Sales,Sales and Marketing
292,287,European Sales Manager,adventure-works\amy0,3,Sales,Sales and Marketing
293,288,Sales Representative,adventure-works\rachel0,3,Sales,Sales and Marketing
294,289,Sales Representative,adventure-works\jae0,3,Sales,Sales and Marketing


In [75]:

salespersonMerge = salesperson
personMerge = person
salespersonMerge = salespersonMerge[['businessentityid','territoryid']]
personMerge = personMerge[['businessentityid','firstname','lastname']]
#dimemployee = dimemployee.merge(salespersonMerge, on="businessentityid", how="left")


In [76]:
dimemployee = dimemployee.merge(salespersonMerge, on="businessentityid", how="left")

In [77]:
dimemployee = dimemployee.merge(personMerge, on="businessentityid", how="left")

In [78]:
dimemployee = dimemployee.rename(columns={"businessentityid": "employeekey",'name':'departmentname'})

In [79]:
#DIMENCION EMPLADO COMPLETA
dimemployee

,employeekey,jobtitle,loginid,departmentid,departmentname,groupname,territoryid,firstname,lastname
0,1,Chief Executive Officer,adventure-works\ken0,16,Executive,Executive General and Administration,NaN,Ken,Sánchez
1,2,Vice President of Engineering,adventure-works\terri0,1,Engineering,Research and Development,NaN,Terri,Duffy
2,3,Engineering Manager,adventure-works\roberto0,1,Engineering,Research and Development,NaN,Roberto,Tamburello
3,4,Senior Tool Designer,adventure-works\rob0,1,Engineering,Research and Development,NaN,Rob,Walters
4,4,Senior Tool Designer,adventure-works\rob0,2,Tool Design,Research and Development,NaN,Rob,Walters
...,...,...,...,...,...,...,...,...,...
291,286,Sales Representative,adventure-works\lynn0,3,Sales,Sales and Marketing,9.0,Lynn,Tsoflias
292,287,European Sales Manager,adventure-works\amy0,3,Sales,Sales and Marketing,NaN,Amy,Alberts
293,288,Sales Representative,adventure-works\rachel0,3,Sales,Sales and Marketing,8.0,Rachel,Valdez
294,289,Sales Representative,adventure-works\jae0,3,Sales,Sales and Marketing,10.0,Jae,Pak


In [65]:
#dimencion producto
dimproduct = product[['productsubcategoryid','name','color', 'size','listprice']]
subcategoryMerge = subcategory.rename(columns={'name':'productsubcategoryname'})
dimproduct = product.merge(subcategoryMerge, on = "productsubcategoryid", how="left")
dimproduct

,productid,name,productnumber,makeflag,finishedgoodsflag,color,safetystocklevel,reorderpoint,standardcost,listprice,...,productmodelid,sellstartdate,sellenddate,discontinueddate,rowguid_x,modifieddate_x,productcategoryid,productsubcategoryname,rowguid_y,modifieddate_y
0,1,Adjustable Race,AR-5381,False,False,None,1000,750,0.0000,0.00,...,NaN,2008-04-30 00:00:00+00:00,NaT,None,694215b7-08f7-4c0d-acb1-d734ba44c0c8,2014-02-08 10:01:36.827000+00:00,NaN,NaN,NaN,NaT
1,2,Bearing Ball,BA-8327,False,False,None,1000,750,0.0000,0.00,...,NaN,2008-04-30 00:00:00+00:00,NaT,None,58ae3c20-4f3a-4749-a7d4-d568806cc537,2014-02-08 10:01:36.827000+00:00,NaN,NaN,NaN,NaT
2,3,BB Ball Bearing,BE-2349,True,False,None,800,600,0.0000,0.00,...,NaN,2008-04-30 00:00:00+00:00,NaT,None,9c21aed2-5bfa-4f18-bcb8-f11638dc2e4e,2014-02-08 10:01:36.827000+00:00,NaN,NaN,NaN,NaT
3,4,Headset Ball Bearings,BE-2908,False,False,None,800,600,0.0000,0.00,...,NaN,2008-04-30 00:00:00+00:00,NaT,None,ecfed6cb-51ff-49b5-b06c-7d8ac834db8b,2014-02-08 10:01:36.827000+00:00,NaN,NaN,NaN,NaT
4,316,Blade,BL-2036,True,False,None,800,600,0.0000,0.00,...,NaN,2008-04-30 00:00:00+00:00,NaT,None,e73e9750-603b-4131-89f5-3dd15ed5ff80,2014-02-08 10:01:36.827000+00:00,NaN,NaN,NaN,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499,995,ML Bottom Bracket,BB-8107,True,True,None,500,375,44.9506,101.24,...,96.0,2013-05-30 00:00:00+00:00,NaT,None,71ab847f-d091-42d6-b735-7b0c2d82fc84,2014-02-08 10:01:36.827000+00:00,2.0,Bottom Brackets,a9e54089-8a1e-4cf5-8646-e3801f685934,2008-04-30 00:00:00+00:00
500,996,HL Bottom Bracket,BB-9108,True,True,None,500,375,53.9416,121.49,...,97.0,2013-05-30 00:00:00+00:00,NaT,None,230c47c5-08b2-4ce3-b706-69c0bdd62965,2014-02-08 10:01:36.827000+00:00,2.0,Bottom Brackets,a9e54089-8a1e-4cf5-8646-e3801f685934,2008-04-30 00:00:00+00:00
501,997,"Road-750 Black, 44",BK-R19B-44,True,True,Black,100,75,343.6496,539.99,...,31.0,2013-05-30 00:00:00+00:00,NaT,None,44ce4802-409f-43ab-9b27-ca53421805be,2014-02-08 10:01:36.827000+00:00,1.0,Road Bikes,000310c0-bcc8-42c4-b0c3-45ae611af06b,2008-04-30 00:00:00+00:00
502,998,"Road-750 Black, 48",BK-R19B-48,True,True,Black,100,75,343.6496,539.99,...,31.0,2013-05-30 00:00:00+00:00,NaT,None,3de9a212-1d49-40b6-b10a-f564d981dbde,2014-02-08 10:01:36.827000+00:00,1.0,Road Bikes,000310c0-bcc8-42c4-b0c3-45ae611af06b,2008-04-30 00:00:00+00:00


In [ ]:
categoryMerge = category.rename(columns={'name':'productcategoryname'})
dimproduct = dimproduct.merge(categoryMerge, on="productcategoryid", how="left")


,productid,name,productnumber,makeflag,finishedgoodsflag,color,safetystocklevel,reorderpoint,standardcost,listprice,...,discontinueddate,rowguid_x,modifieddate_x,productcategoryid,productsubcategoryname,rowguid_y,modifieddate_y,productcategoryname,rowguid,modifieddate
0,1,Adjustable Race,AR-5381,False,False,None,1000,750,0.0000,0.00,...,None,694215b7-08f7-4c0d-acb1-d734ba44c0c8,2014-02-08 10:01:36.827000+00:00,NaN,NaN,NaN,NaT,NaN,NaN,NaT
1,2,Bearing Ball,BA-8327,False,False,None,1000,750,0.0000,0.00,...,None,58ae3c20-4f3a-4749-a7d4-d568806cc537,2014-02-08 10:01:36.827000+00:00,NaN,NaN,NaN,NaT,NaN,NaN,NaT
2,3,BB Ball Bearing,BE-2349,True,False,None,800,600,0.0000,0.00,...,None,9c21aed2-5bfa-4f18-bcb8-f11638dc2e4e,2014-02-08 10:01:36.827000+00:00,NaN,NaN,NaN,NaT,NaN,NaN,NaT
3,4,Headset Ball Bearings,BE-2908,False,False,None,800,600,0.0000,0.00,...,None,ecfed6cb-51ff-49b5-b06c-7d8ac834db8b,2014-02-08 10:01:36.827000+00:00,NaN,NaN,NaN,NaT,NaN,NaN,NaT
4,316,Blade,BL-2036,True,False,None,800,600,0.0000,0.00,...,None,e73e9750-603b-4131-89f5-3dd15ed5ff80,2014-02-08 10:01:36.827000+00:00,NaN,NaN,NaN,NaT,NaN,NaN,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499,995,ML Bottom Bracket,BB-8107,True,True,None,500,375,44.9506,101.24,...,None,71ab847f-d091-42d6-b735-7b0c2d82fc84,2014-02-08 10:01:36.827000+00:00,2.0,Bottom Brackets,a9e54089-8a1e-4cf5-8646-e3801f685934,2008-04-30 00:00:00+00:00,Components,c657828d-d808-4aba-91a3-af2ce02300e9,2008-04-30 00:00:00+00:00
500,996,HL Bottom Bracket,BB-9108,True,True,None,500,375,53.9416,121.49,...,None,230c47c5-08b2-4ce3-b706-69c0bdd62965,2014-02-08 10:01:36.827000+00:00,2.0,Bottom Brackets,a9e54089-8a1e-4cf5-8646-e3801f685934,2008-04-30 00:00:00+00:00,Components,c657828d-d808-4aba-91a3-af2ce02300e9,2008-04-30 00:00:00+00:00
501,997,"Road-750 Black, 44",BK-R19B-44,True,True,Black,100,75,343.6496,539.99,...,None,44ce4802-409f-43ab-9b27-ca53421805be,2014-02-08 10:01:36.827000+00:00,1.0,Road Bikes,000310c0-bcc8-42c4-b0c3-45ae611af06b,2008-04-30 00:00:00+00:00,Bikes,cfbda25c-df71-47a7-b81b-64ee161aa37c,2008-04-30 00:00:00+00:00
502,998,"Road-750 Black, 48",BK-R19B-48,True,True,Black,100,75,343.6496,539.99,...,None,3de9a212-1d49-40b6-b10a-f564d981dbde,2014-02-08 10:01:36.827000+00:00,1.0,Road Bikes,000310c0-bcc8-42c4-b0c3-45ae611af06b,2008-04-30 00:00:00+00:00,Bikes,cfbda25c-df71-47a7-b81b-64ee161aa37c,2008-04-30 00:00:00+00:00


In [80]:
dimproduct = dimproduct[[
    "productid",
    "name",
    "color",
    "size",
    "productsubcategoryid",
    "productsubcategoryname",
    "productcategoryid",
    "productcategoryname",
    "listprice"
]].drop_duplicates()

In [83]:
#DIMENSION PRODUCTO
dimproduct

,productid,name,color,size,productsubcategoryid,productsubcategoryname,productcategoryid,productcategoryname,listprice
0,1,Adjustable Race,None,None,NaN,NaN,NaN,NaN,0.00
1,2,Bearing Ball,None,None,NaN,NaN,NaN,NaN,0.00
2,3,BB Ball Bearing,None,None,NaN,NaN,NaN,NaN,0.00
3,4,Headset Ball Bearings,None,None,NaN,NaN,NaN,NaN,0.00
4,316,Blade,None,None,NaN,NaN,NaN,NaN,0.00
...,...,...,...,...,...,...,...,...,...
499,995,ML Bottom Bracket,None,None,5.0,Bottom Brackets,2.0,Components,101.24
500,996,HL Bottom Bracket,None,None,5.0,Bottom Brackets,2.0,Components,121.49
501,997,"Road-750 Black, 44",Black,44,2.0,Road Bikes,1.0,Bikes,539.99
502,998,"Road-750 Black, 48",Black,48,2.0,Road Bikes,1.0,Bikes,539.99


In [90]:
#DIMENSION PROMOCION
dimpromotion = (
    specialoffer
    .merge(specialofferproduct, on="specialofferid", how="left")
)[[
    "specialofferid",
    "description",
    "discountpct",
    "startdate",
    "enddate",
    "productid",
    "category",
    "type",
    "minqty",
    "maxqty"
]].drop_duplicates()

In [91]:
dimpromotion

,specialofferid,description,discountpct,startdate,enddate,productid,category,type,minqty,maxqty
0,1,No Discount,0.0,2011-05-01 00:00:00+00:00,2014-11-30 00:00:00+00:00,680.0,No Discount,No Discount,0,NaN
1,1,No Discount,0.0,2011-05-01 00:00:00+00:00,2014-11-30 00:00:00+00:00,706.0,No Discount,No Discount,0,NaN
2,1,No Discount,0.0,2011-05-01 00:00:00+00:00,2014-11-30 00:00:00+00:00,707.0,No Discount,No Discount,0,NaN
3,1,No Discount,0.0,2011-05-01 00:00:00+00:00,2014-11-30 00:00:00+00:00,708.0,No Discount,No Discount,0,NaN
4,1,No Discount,0.0,2011-05-01 00:00:00+00:00,2014-11-30 00:00:00+00:00,709.0,No Discount,No Discount,0,NaN
...,...,...,...,...,...,...,...,...,...,...
534,16,Mountain-500 Silver Clearance Sale,0.4,2014-03-31 00:00:00+00:00,2014-05-30 00:00:00+00:00,984.0,Reseller,Discontinued Product,0,NaN
535,16,Mountain-500 Silver Clearance Sale,0.4,2014-03-31 00:00:00+00:00,2014-05-30 00:00:00+00:00,985.0,Reseller,Discontinued Product,0,NaN
536,16,Mountain-500 Silver Clearance Sale,0.4,2014-03-31 00:00:00+00:00,2014-05-30 00:00:00+00:00,986.0,Reseller,Discontinued Product,0,NaN
537,16,Mountain-500 Silver Clearance Sale,0.4,2014-03-31 00:00:00+00:00,2014-05-30 00:00:00+00:00,987.0,Reseller,Discontinued Product,0,NaN


In [92]:
#DIM SALES TERRITORY
dimsalesterritory = salesterritory[[
    "territoryid",
    "name",
    "countryregioncode",
    "group"
]].drop_duplicates()
dimsalesterritory

,territoryid,name,countryregioncode,group
0,1,Northwest,US,North America
1,2,Northeast,US,North America
2,3,Central,US,North America
3,4,Southwest,US,North America
4,5,Southeast,US,North America
5,6,Canada,CA,North America
6,7,France,FR,Europe
7,8,Germany,DE,Europe
8,9,Australia,AU,Pacific
9,10,United Kingdom,GB,Europe


In [129]:
#HECHO VENTAS POR INTERNET
internet_sales = salesorderheader[salesorderheader["onlineorderflag"] == True]

In [130]:
factInternet = (internet_sales.merge(salesorderdetail, on="salesorderid"))
factInternet

,salesorderid,revisionnumber,orderdate,duedate,shipdate,status,onlineorderflag,salesordernumber,purchaseordernumber,accountnumber,...,salesorderdetailid,carriertrackingnumber,orderqty,productid,specialofferid,unitprice,unitpricediscount,linetotal,rowguid_y,modifieddate_y
0,43697,8,2011-05-31 00:00:00+00:00,2011-06-12 00:00:00+00:00,2011-06-07 00:00:00+00:00,5,True,SO43697,None,10-4030-021768,...,353,None,1,749,1,3578.2700,0.0,3578.2700,ca258bf8-1f52-4ae6-8e8f-5439ae0c9509,2011-05-31 00:00:00+00:00
1,43698,8,2011-05-31 00:00:00+00:00,2011-06-12 00:00:00+00:00,2011-06-07 00:00:00+00:00,5,True,SO43698,None,10-4030-028389,...,354,None,1,773,1,3399.9900,0.0,3399.9900,a2d7730f-d2d0-4c47-8f36-fbd316680cef,2011-05-31 00:00:00+00:00
2,43699,8,2011-05-31 00:00:00+00:00,2011-06-12 00:00:00+00:00,2011-06-07 00:00:00+00:00,5,True,SO43699,None,10-4030-025863,...,355,None,1,773,1,3399.9900,0.0,3399.9900,2543c7bf-679b-47f3-a04f-eb5da849ef32,2011-05-31 00:00:00+00:00
3,43700,8,2011-05-31 00:00:00+00:00,2011-06-12 00:00:00+00:00,2011-06-07 00:00:00+00:00,5,True,SO43700,None,10-4030-014501,...,356,None,1,767,1,699.0982,0.0,699.0982,6d43204a-dba1-4013-bc4d-0273baa703c9,2011-05-31 00:00:00+00:00
4,43701,8,2011-05-31 00:00:00+00:00,2011-06-12 00:00:00+00:00,2011-06-07 00:00:00+00:00,5,True,SO43701,None,10-4030-011003,...,357,None,1,773,1,3399.9900,0.0,3399.9900,5ff3b05f-57f0-4885-8e7b-64c4ad71aeb5,2011-05-31 00:00:00+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60393,75122,8,2014-06-30 00:00:00+00:00,2014-07-12 00:00:00+00:00,2014-07-07 00:00:00+00:00,5,True,SO75122,None,10-4030-015868,...,121313,None,1,878,1,21.9800,0.0,21.9800,8cad6675-18cc-4f47-8287-97b41a8ee47d,2014-06-30 00:00:00+00:00
60394,75122,8,2014-06-30 00:00:00+00:00,2014-07-12 00:00:00+00:00,2014-07-07 00:00:00+00:00,5,True,SO75122,None,10-4030-015868,...,121314,None,1,712,1,8.9900,0.0,8.9900,84f1c363-1c50-4442-be16-541c59b6e12c,2014-06-30 00:00:00+00:00
60395,75123,8,2014-06-30 00:00:00+00:00,2014-07-12 00:00:00+00:00,2014-07-07 00:00:00+00:00,5,True,SO75123,None,10-4030-018759,...,121315,None,1,878,1,21.9800,0.0,21.9800,c18b6476-429f-4bb1-828e-2be5f82a0a51,2014-06-30 00:00:00+00:00
60396,75123,8,2014-06-30 00:00:00+00:00,2014-07-12 00:00:00+00:00,2014-07-07 00:00:00+00:00,5,True,SO75123,None,10-4030-018759,...,121316,None,1,879,1,159.0000,0.0,159.0000,75a89c6a-c60a-47ea-8a52-b52a9c435b64,2014-06-30 00:00:00+00:00


In [131]:
factInternet = factInternet.merge(dimproduct, on="productid", how="left")
factInternet

,salesorderid,revisionnumber,orderdate,duedate,shipdate,status,onlineorderflag,salesordernumber,purchaseordernumber,accountnumber,...,rowguid_y,modifieddate_y,name,color,size,productsubcategoryid,productsubcategoryname,productcategoryid,productcategoryname,listprice
0,43697,8,2011-05-31 00:00:00+00:00,2011-06-12 00:00:00+00:00,2011-06-07 00:00:00+00:00,5,True,SO43697,None,10-4030-021768,...,ca258bf8-1f52-4ae6-8e8f-5439ae0c9509,2011-05-31 00:00:00+00:00,"Road-150 Red, 62",Red,62,2.0,Road Bikes,1.0,Bikes,3578.27
1,43698,8,2011-05-31 00:00:00+00:00,2011-06-12 00:00:00+00:00,2011-06-07 00:00:00+00:00,5,True,SO43698,None,10-4030-028389,...,a2d7730f-d2d0-4c47-8f36-fbd316680cef,2011-05-31 00:00:00+00:00,"Mountain-100 Silver, 44",Silver,44,1.0,Mountain Bikes,1.0,Bikes,3399.99
2,43699,8,2011-05-31 00:00:00+00:00,2011-06-12 00:00:00+00:00,2011-06-07 00:00:00+00:00,5,True,SO43699,None,10-4030-025863,...,2543c7bf-679b-47f3-a04f-eb5da849ef32,2011-05-31 00:00:00+00:00,"Mountain-100 Silver, 44",Silver,44,1.0,Mountain Bikes,1.0,Bikes,3399.99
3,43700,8,2011-05-31 00:00:00+00:00,2011-06-12 00:00:00+00:00,2011-06-07 00:00:00+00:00,5,True,SO43700,None,10-4030-014501,...,6d43204a-dba1-4013-bc4d-0273baa703c9,2011-05-31 00:00:00+00:00,"Road-650 Black, 62",Black,62,2.0,Road Bikes,1.0,Bikes,782.99
4,43701,8,2011-05-31 00:00:00+00:00,2011-06-12 00:00:00+00:00,2011-06-07 00:00:00+00:00,5,True,SO43701,None,10-4030-011003,...,5ff3b05f-57f0-4885-8e7b-64c4ad71aeb5,2011-05-31 00:00:00+00:00,"Mountain-100 Silver, 44",Silver,44,1.0,Mountain Bikes,1.0,Bikes,3399.99
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60393,75122,8,2014-06-30 00:00:00+00:00,2014-07-12 00:00:00+00:00,2014-07-07 00:00:00+00:00,5,True,SO75122,None,10-4030-015868,...,8cad6675-18cc-4f47-8287-97b41a8ee47d,2014-06-30 00:00:00+00:00,Fender Set - Mountain,None,None,30.0,Fenders,4.0,Accessories,21.98
60394,75122,8,2014-06-30 00:00:00+00:00,2014-07-12 00:00:00+00:00,2014-07-07 00:00:00+00:00,5,True,SO75122,None,10-4030-015868,...,84f1c363-1c50-4442-be16-541c59b6e12c,2014-06-30 00:00:00+00:00,AWC Logo Cap,Multi,None,19.0,Caps,3.0,Clothing,8.99
60395,75123,8,2014-06-30 00:00:00+00:00,2014-07-12 00:00:00+00:00,2014-07-07 00:00:00+00:00,5,True,SO75123,None,10-4030-018759,...,c18b6476-429f-4bb1-828e-2be5f82a0a51,2014-06-30 00:00:00+00:00,Fender Set - Mountain,None,None,30.0,Fenders,4.0,Accessories,21.98
60396,75123,8,2014-06-30 00:00:00+00:00,2014-07-12 00:00:00+00:00,2014-07-07 00:00:00+00:00,5,True,SO75123,None,10-4030-018759,...,75a89c6a-c60a-47ea-8a52-b52a9c435b64,2014-06-30 00:00:00+00:00,All-Purpose Bike Stand,None,None,27.0,Bike Stands,4.0,Accessories,159.00


In [132]:
mergedimecustomer = dimcustomer
mergedimecustomer = mergedimecustomer.rename(columns={'territoryid': 'territoryidCustomer'})
factInternet = factInternet.merge(mergedimecustomer, on="customerid", how="left")
factInternet

,salesorderid,revisionnumber,orderdate,duedate,shipdate,status,onlineorderflag,salesordernumber,purchaseordernumber,accountnumber,...,productcategoryid,productcategoryname,listprice,firstname,lastname,addressline1,city,stateprovinceid,countryregioncode,territoryidCustomer
0,43697,8,2011-05-31 00:00:00+00:00,2011-06-12 00:00:00+00:00,2011-06-07 00:00:00+00:00,5,True,SO43697,None,10-4030-021768,...,1.0,Bikes,3578.27,Cole,Watson,601 Asilomar Dr.,Metchosin,7,CA,6
1,43698,8,2011-05-31 00:00:00+00:00,2011-06-12 00:00:00+00:00,2011-06-07 00:00:00+00:00,5,True,SO43698,None,10-4030-028389,...,1.0,Bikes,3399.99,Rachael,Martinez,"14, avenue du Port",Pantin,179,FR,7
2,43699,8,2011-05-31 00:00:00+00:00,2011-06-12 00:00:00+00:00,2011-06-07 00:00:00+00:00,5,True,SO43699,None,10-4030-025863,...,1.0,Bikes,3399.99,Sydney,Wright,4193 E. 28th Street,Lebanon,58,US,1
3,43700,8,2011-05-31 00:00:00+00:00,2011-06-12 00:00:00+00:00,2011-06-07 00:00:00+00:00,5,True,SO43700,None,10-4030-014501,...,1.0,Bikes,782.99,Ruben,Prasad,249 Alexander Pl.,Beverly Hills,9,US,4
4,43701,8,2011-05-31 00:00:00+00:00,2011-06-12 00:00:00+00:00,2011-06-07 00:00:00+00:00,5,True,SO43701,None,10-4030-011003,...,1.0,Bikes,3399.99,Christy,Zhu,1825 Village Pl.,North Ryde,50,AU,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60454,75122,8,2014-06-30 00:00:00+00:00,2014-07-12 00:00:00+00:00,2014-07-07 00:00:00+00:00,5,True,SO75122,None,10-4030-015868,...,4.0,Accessories,21.98,Caleb,Lal,4037 San View Way,Sooke,7,CA,6
60455,75122,8,2014-06-30 00:00:00+00:00,2014-07-12 00:00:00+00:00,2014-07-07 00:00:00+00:00,5,True,SO75122,None,10-4030-015868,...,3.0,Clothing,8.99,Caleb,Lal,4037 San View Way,Sooke,7,CA,6
60456,75123,8,2014-06-30 00:00:00+00:00,2014-07-12 00:00:00+00:00,2014-07-07 00:00:00+00:00,5,True,SO75123,None,10-4030-018759,...,4.0,Accessories,21.98,Devin,Phillips,2742 Cincerto Circle,Sooke,7,CA,6
60457,75123,8,2014-06-30 00:00:00+00:00,2014-07-12 00:00:00+00:00,2014-07-07 00:00:00+00:00,5,True,SO75123,None,10-4030-018759,...,4.0,Accessories,159.00,Devin,Phillips,2742 Cincerto Circle,Sooke,7,CA,6


In [134]:
factInternet = factInternet.merge(dimpromotion, on="productid", how="left")
factInternet

,salesorderid,revisionnumber,orderdate,duedate,shipdate,status,onlineorderflag,salesordernumber,purchaseordernumber,accountnumber,...,maxqty_x,specialofferid,description_y,discountpct_y,startdate_y,enddate_y,category_y,type_y,minqty_y,maxqty_y
0,43697,8,2011-05-31 00:00:00+00:00,2011-06-12 00:00:00+00:00,2011-06-07 00:00:00+00:00,5,True,SO43697,None,10-4030-021768,...,NaN,1,No Discount,0.00,2011-05-01 00:00:00+00:00,2014-11-30 00:00:00+00:00,No Discount,No Discount,0,NaN
1,43698,8,2011-05-31 00:00:00+00:00,2011-06-12 00:00:00+00:00,2011-06-07 00:00:00+00:00,5,True,SO43698,None,10-4030-028389,...,NaN,1,No Discount,0.00,2011-05-01 00:00:00+00:00,2014-11-30 00:00:00+00:00,No Discount,No Discount,0,NaN
2,43698,8,2011-05-31 00:00:00+00:00,2011-06-12 00:00:00+00:00,2011-06-07 00:00:00+00:00,5,True,SO43698,None,10-4030-028389,...,NaN,2,Volume Discount 11 to 14,0.02,2011-05-31 00:00:00+00:00,2014-05-30 00:00:00+00:00,Reseller,Volume Discount,11,14.0
3,43698,8,2011-05-31 00:00:00+00:00,2011-06-12 00:00:00+00:00,2011-06-07 00:00:00+00:00,5,True,SO43698,None,10-4030-028389,...,NaN,7,Mountain-100 Clearance Sale,0.35,2012-04-13 00:00:00+00:00,2012-05-29 00:00:00+00:00,Reseller,Discontinued Product,0,NaN
4,43698,8,2011-05-31 00:00:00+00:00,2011-06-12 00:00:00+00:00,2011-06-07 00:00:00+00:00,5,True,SO43698,None,10-4030-028389,...,14.0,1,No Discount,0.00,2011-05-01 00:00:00+00:00,2014-11-30 00:00:00+00:00,No Discount,No Discount,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
478845,75123,8,2014-06-30 00:00:00+00:00,2014-07-12 00:00:00+00:00,2014-07-07 00:00:00+00:00,5,True,SO75123,None,10-4030-018759,...,24.0,4,Volume Discount 25 to 40,0.10,2011-05-31 00:00:00+00:00,2014-05-30 00:00:00+00:00,Reseller,Volume Discount,25,40.0
478846,75123,8,2014-06-30 00:00:00+00:00,2014-07-12 00:00:00+00:00,2014-07-07 00:00:00+00:00,5,True,SO75123,None,10-4030-018759,...,40.0,1,No Discount,0.00,2011-05-01 00:00:00+00:00,2014-11-30 00:00:00+00:00,No Discount,No Discount,0,NaN
478847,75123,8,2014-06-30 00:00:00+00:00,2014-07-12 00:00:00+00:00,2014-07-07 00:00:00+00:00,5,True,SO75123,None,10-4030-018759,...,40.0,2,Volume Discount 11 to 14,0.02,2011-05-31 00:00:00+00:00,2014-05-30 00:00:00+00:00,Reseller,Volume Discount,11,14.0
478848,75123,8,2014-06-30 00:00:00+00:00,2014-07-12 00:00:00+00:00,2014-07-07 00:00:00+00:00,5,True,SO75123,None,10-4030-018759,...,40.0,3,Volume Discount 15 to 24,0.05,2011-05-31 00:00:00+00:00,2014-05-30 00:00:00+00:00,Reseller,Volume Discount,15,24.0


In [135]:
factInternet = factInternet.merge(dimsalesterritory, on="territoryid", how="left")
factInternet

,salesorderid,revisionnumber,orderdate,duedate,shipdate,status,onlineorderflag,salesordernumber,purchaseordernumber,accountnumber,...,discountpct_y,startdate_y,enddate_y,category_y,type_y,minqty_y,maxqty_y,name_y,countryregioncode_y,group
0,43697,8,2011-05-31 00:00:00+00:00,2011-06-12 00:00:00+00:00,2011-06-07 00:00:00+00:00,5,True,SO43697,None,10-4030-021768,...,0.00,2011-05-01 00:00:00+00:00,2014-11-30 00:00:00+00:00,No Discount,No Discount,0,NaN,Canada,CA,North America
1,43698,8,2011-05-31 00:00:00+00:00,2011-06-12 00:00:00+00:00,2011-06-07 00:00:00+00:00,5,True,SO43698,None,10-4030-028389,...,0.00,2011-05-01 00:00:00+00:00,2014-11-30 00:00:00+00:00,No Discount,No Discount,0,NaN,France,FR,Europe
2,43698,8,2011-05-31 00:00:00+00:00,2011-06-12 00:00:00+00:00,2011-06-07 00:00:00+00:00,5,True,SO43698,None,10-4030-028389,...,0.02,2011-05-31 00:00:00+00:00,2014-05-30 00:00:00+00:00,Reseller,Volume Discount,11,14.0,France,FR,Europe
3,43698,8,2011-05-31 00:00:00+00:00,2011-06-12 00:00:00+00:00,2011-06-07 00:00:00+00:00,5,True,SO43698,None,10-4030-028389,...,0.35,2012-04-13 00:00:00+00:00,2012-05-29 00:00:00+00:00,Reseller,Discontinued Product,0,NaN,France,FR,Europe
4,43698,8,2011-05-31 00:00:00+00:00,2011-06-12 00:00:00+00:00,2011-06-07 00:00:00+00:00,5,True,SO43698,None,10-4030-028389,...,0.00,2011-05-01 00:00:00+00:00,2014-11-30 00:00:00+00:00,No Discount,No Discount,0,NaN,France,FR,Europe
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
478845,75123,8,2014-06-30 00:00:00+00:00,2014-07-12 00:00:00+00:00,2014-07-07 00:00:00+00:00,5,True,SO75123,None,10-4030-018759,...,0.10,2011-05-31 00:00:00+00:00,2014-05-30 00:00:00+00:00,Reseller,Volume Discount,25,40.0,Canada,CA,North America
478846,75123,8,2014-06-30 00:00:00+00:00,2014-07-12 00:00:00+00:00,2014-07-07 00:00:00+00:00,5,True,SO75123,None,10-4030-018759,...,0.00,2011-05-01 00:00:00+00:00,2014-11-30 00:00:00+00:00,No Discount,No Discount,0,NaN,Canada,CA,North America
478847,75123,8,2014-06-30 00:00:00+00:00,2014-07-12 00:00:00+00:00,2014-07-07 00:00:00+00:00,5,True,SO75123,None,10-4030-018759,...,0.02,2011-05-31 00:00:00+00:00,2014-05-30 00:00:00+00:00,Reseller,Volume Discount,11,14.0,Canada,CA,North America
478848,75123,8,2014-06-30 00:00:00+00:00,2014-07-12 00:00:00+00:00,2014-07-07 00:00:00+00:00,5,True,SO75123,None,10-4030-018759,...,0.05,2011-05-31 00:00:00+00:00,2014-05-30 00:00:00+00:00,Reseller,Volume Discount,15,24.0,Canada,CA,North America


In [141]:
# Generar claves de fecha
factInternet["orderdatekey"] = pd.to_datetime(factInternet["orderdate"]).dt.strftime("%Y%m%d").astype(int)
factInternet["duedatekey"] = pd.to_datetime(factInternet["duedate"]).dt.strftime("%Y%m%d").astype(int)
factInternet["shipdatekey"] = pd.to_datetime(factInternet["shipdate"]).dt.strftime("%Y%m%d").astype(int)



In [146]:
factInternet = factInternet[[
    "salesorderid",
    "productid",
    "customerid",
    "territoryid",
    "orderqty",
    "unitprice",
    "unitpricediscount",
    "freight",
    "taxamt",
    "totaldue",
    "orderdatekey",
    "duedatekey",
    "shipdatekey"
]]
factInternet

,salesorderid,productid,customerid,territoryid,orderqty,unitprice,unitpricediscount,freight,taxamt,totaldue,orderdatekey,duedatekey,shipdatekey
0,43697,749,21768,6,1,3578.27,0.0,89.4568,286.2616,3953.9883,20110531,20110612,20110607
1,43698,773,28389,7,1,3399.99,0.0,84.9998,271.9992,3756.9890,20110531,20110612,20110607
2,43698,773,28389,7,1,3399.99,0.0,84.9998,271.9992,3756.9890,20110531,20110612,20110607
3,43698,773,28389,7,1,3399.99,0.0,84.9998,271.9992,3756.9890,20110531,20110612,20110607
4,43698,773,28389,7,1,3399.99,0.0,84.9998,271.9992,3756.9890,20110531,20110612,20110607
...,...,...,...,...,...,...,...,...,...,...,...,...,...
478845,75123,712,18759,6,1,8.99,0.0,4.7493,15.1976,209.9169,20140630,20140712,20140707
478846,75123,712,18759,6,1,8.99,0.0,4.7493,15.1976,209.9169,20140630,20140712,20140707
478847,75123,712,18759,6,1,8.99,0.0,4.7493,15.1976,209.9169,20140630,20140712,20140707
478848,75123,712,18759,6,1,8.99,0.0,4.7493,15.1976,209.9169,20140630,20140712,20140707


In [159]:
#HECHO VENTAS POR REVENDEDORES  &(salesorderheader["storeid"].notnull())
customerMergeRevendedores = customer
salespersonCopiaTerritory = salesperson[['territoryid', 'businessentityid']]
customerMergeRevendedores = customerMergeRevendedores[['customerid','storeid']]
reseller_sales = salesorderheader[(salesorderheader["onlineorderflag"] == False)]
reseller_sales = reseller_sales.merge(customerMergeRevendedores, on="customerid", how="left")
reseller_sales = reseller_sales.merge(salespersonCopiaTerritory, on="territoryid", how="left")



In [160]:
reseller_sales = reseller_sales[(reseller_sales["storeid"].notnull())]
reseller_sales

,salesorderid,revisionnumber,orderdate,duedate,shipdate,status,onlineorderflag,salesordernumber,purchaseordernumber,accountnumber,...,currencyrateid,subtotal,taxamt,freight,totaldue,comment,rowguid,modifieddate,storeid,businessentityid
0,43659,8,2011-05-31 00:00:00+00:00,2011-06-12 00:00:00+00:00,2011-06-07 00:00:00+00:00,5,False,SO43659,PO522145787,10-4020-000676,...,NaN,20565.6210,1971.5149,616.0984,23153.2340,None,79b65321-39ca-4115-9cba-8fe0903e12e6,2011-06-07 00:00:00+00:00,1046.0,279
1,43660,8,2011-05-31 00:00:00+00:00,2011-06-12 00:00:00+00:00,2011-06-07 00:00:00+00:00,5,False,SO43660,PO18850127500,10-4020-000117,...,NaN,1294.2529,124.2483,38.8276,1457.3289,None,738dc42d-d03b-48a1-9822-f95a67ea7389,2011-06-07 00:00:00+00:00,722.0,279
2,43661,8,2011-05-31 00:00:00+00:00,2011-06-12 00:00:00+00:00,2011-06-07 00:00:00+00:00,5,False,SO43661,PO18473189620,10-4020-000442,...,4.0,32726.4790,3153.7695,985.5530,36865.8000,None,d91b9131-18a4-4a11-bc3a-90b6f53e9d74,2011-06-07 00:00:00+00:00,852.0,278
3,43661,8,2011-05-31 00:00:00+00:00,2011-06-12 00:00:00+00:00,2011-06-07 00:00:00+00:00,5,False,SO43661,PO18473189620,10-4020-000442,...,4.0,32726.4790,3153.7695,985.5530,36865.8000,None,d91b9131-18a4-4a11-bc3a-90b6f53e9d74,2011-06-07 00:00:00+00:00,852.0,282
4,43662,8,2011-05-31 00:00:00+00:00,2011-06-12 00:00:00+00:00,2011-06-07 00:00:00+00:00,5,False,SO43662,PO18444174044,10-4020-000227,...,4.0,28832.5300,2775.1646,867.2389,32474.9320,None,4a1ecfc0-cc3a-4740-b028-1c50bb48711c,2011-06-07 00:00:00+00:00,1418.0,278
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6316,71949,8,2014-05-01 00:00:00+00:00,2014-05-13 00:00:00+00:00,2014-05-08 00:00:00+00:00,5,False,SO71949,PO9483195097,10-4020-000054,...,NaN,34123.6680,3253.9468,1016.8583,38394.4730,None,0ffd145d-43b3-4a8d-b2e9-2e52e570820f,2014-05-08 00:00:00+00:00,1356.0,275
6317,71950,8,2014-05-01 00:00:00+00:00,2014-05-13 00:00:00+00:00,2014-05-08 00:00:00+00:00,5,False,SO71950,PO9744193484,10-4020-000098,...,NaN,3131.9340,297.1382,92.8557,3521.9280,None,bfe09bc4-a7db-48cd-841e-572715972a20,2014-05-08 00:00:00+00:00,734.0,279
6318,71951,8,2014-05-01 00:00:00+00:00,2014-05-13 00:00:00+00:00,2014-05-08 00:00:00+00:00,5,False,SO71951,PO9106170008,10-4020-000602,...,NaN,1502.9760,143.1099,44.7218,1690.8077,None,2c020885-ad87-4563-a7ab-6e9595a75858,2014-05-08 00:00:00+00:00,654.0,279
6319,71952,8,2014-05-01 00:00:00+00:00,2014-05-13 00:00:00+00:00,2014-05-08 00:00:00+00:00,5,False,SO71952,PO9715163911,10-4020-000490,...,NaN,67059.6300,6573.0030,2054.0635,75686.7000,None,4b974418-cb6b-4887-8a40-e08cc5054d32,2014-05-08 00:00:00+00:00,1836.0,276


In [177]:
dimemployee = dimemployee.rename(columns={'territoryid': 'territoryidDimEmployee'})

factReseller = (
    reseller_sales
    .merge(salesorderdetail, on="salesorderid")
    .merge(dimproduct, on="productid", how="left")
    .merge(dimemployee, left_on="businessentityid", right_on="employeekey", how="left")
    .merge(dimreseller, on="storeid", how="left")
    .merge(dimpromotion, on="productid", how="left")
    .merge(dimsalesterritory, on="territoryid", how="left")
)
factReseller

,salesorderid,revisionnumber,orderdate,duedate,shipdate,status,onlineorderflag,salesordernumber,purchaseordernumber,accountnumber,...,discountpct,startdate,enddate,category,type,minqty,maxqty,name,countryregioncode,group
0,43659,8,2011-05-31 00:00:00+00:00,2011-06-12 00:00:00+00:00,2011-06-07 00:00:00+00:00,5,False,SO43659,PO522145787,10-4020-000676,...,0.00,2011-05-01 00:00:00+00:00,2014-11-30 00:00:00+00:00,No Discount,No Discount,0,NaN,Southeast,US,North America
1,43659,8,2011-05-31 00:00:00+00:00,2011-06-12 00:00:00+00:00,2011-06-07 00:00:00+00:00,5,False,SO43659,PO522145787,10-4020-000676,...,0.02,2011-05-31 00:00:00+00:00,2014-05-30 00:00:00+00:00,Reseller,Volume Discount,11,14.0,Southeast,US,North America
2,43659,8,2011-05-31 00:00:00+00:00,2011-06-12 00:00:00+00:00,2011-06-07 00:00:00+00:00,5,False,SO43659,PO522145787,10-4020-000676,...,0.35,2012-04-13 00:00:00+00:00,2012-05-29 00:00:00+00:00,Reseller,Discontinued Product,0,NaN,Southeast,US,North America
3,43659,8,2011-05-31 00:00:00+00:00,2011-06-12 00:00:00+00:00,2011-06-07 00:00:00+00:00,5,False,SO43659,PO522145787,10-4020-000676,...,0.00,2011-05-01 00:00:00+00:00,2014-11-30 00:00:00+00:00,No Discount,No Discount,0,NaN,Southeast,US,North America
4,43659,8,2011-05-31 00:00:00+00:00,2011-06-12 00:00:00+00:00,2011-06-07 00:00:00+00:00,5,False,SO43659,PO522145787,10-4020-000676,...,0.02,2011-05-31 00:00:00+00:00,2014-05-30 00:00:00+00:00,Reseller,Volume Discount,11,14.0,Southeast,US,North America
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499409,71952,8,2014-05-01 00:00:00+00:00,2014-05-13 00:00:00+00:00,2014-05-08 00:00:00+00:00,5,False,SO71952,PO9715163911,10-4020-000490,...,0.00,2011-05-01 00:00:00+00:00,2014-11-30 00:00:00+00:00,No Discount,No Discount,0,NaN,Southwest,US,North America
499410,71952,8,2014-05-01 00:00:00+00:00,2014-05-13 00:00:00+00:00,2014-05-08 00:00:00+00:00,5,False,SO71952,PO9715163911,10-4020-000490,...,0.00,2011-05-01 00:00:00+00:00,2014-11-30 00:00:00+00:00,No Discount,No Discount,0,NaN,Southwest,US,North America
499411,71952,8,2014-05-01 00:00:00+00:00,2014-05-13 00:00:00+00:00,2014-05-08 00:00:00+00:00,5,False,SO71952,PO9715163911,10-4020-000490,...,0.40,2014-03-31 00:00:00+00:00,2014-05-30 00:00:00+00:00,Reseller,Discontinued Product,0,NaN,Southwest,US,North America
499412,71952,8,2014-05-01 00:00:00+00:00,2014-05-13 00:00:00+00:00,2014-05-08 00:00:00+00:00,5,False,SO71952,PO9715163911,10-4020-000490,...,0.00,2011-05-01 00:00:00+00:00,2014-11-30 00:00:00+00:00,No Discount,No Discount,0,NaN,Southwest,US,North America


In [180]:

factReseller["orderdatekey"] = pd.to_datetime(factReseller["orderdate"]).dt.strftime("%Y%m%d").astype(int)
factReseller["duedatekey"] = pd.to_datetime(factReseller["duedate"]).dt.strftime("%Y%m%d").astype(int)
factReseller["shipdatekey"] = pd.to_datetime(factReseller["shipdate"]).dt.strftime("%Y%m%d").astype(int)


In [183]:
factReseller = factReseller[[
    "salesorderid",
    "productid",
    "storeid",
    "employeekey",
    "territoryid",
    "orderqty",
    "unitprice",
    "unitpricediscount",
    "freight",
    "taxamt",
    "totaldue",
    "orderdatekey",
    "duedatekey",
    "shipdatekey"
]]
factReseller= factReseller.rename(columns={'employeekey':'businessentityid'})
factReseller

,salesorderid,productid,storeid,businessentityid,territoryid,orderqty,unitprice,unitpricediscount,freight,taxamt,totaldue,orderdatekey,duedatekey,shipdatekey
0,43659,776,1046.0,279,5,1,2024.994,0.0,616.0984,1971.5149,23153.234,20110531,20110612,20110607
1,43659,776,1046.0,279,5,1,2024.994,0.0,616.0984,1971.5149,23153.234,20110531,20110612,20110607
2,43659,776,1046.0,279,5,1,2024.994,0.0,616.0984,1971.5149,23153.234,20110531,20110612,20110607
3,43659,776,1046.0,279,5,1,2024.994,0.0,616.0984,1971.5149,23153.234,20110531,20110612,20110607
4,43659,776,1046.0,279,5,1,2024.994,0.0,616.0984,1971.5149,23153.234,20110531,20110612,20110607
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499409,71952,994,1836.0,281,4,3,32.394,0.0,2054.0635,6573.0030,75686.700,20140501,20140513,20140508
499410,71952,985,1836.0,281,4,3,112.998,0.4,2054.0635,6573.0030,75686.700,20140501,20140513,20140508
499411,71952,985,1836.0,281,4,3,112.998,0.4,2054.0635,6573.0030,75686.700,20140501,20140513,20140508
499412,71952,985,1836.0,281,4,3,112.998,0.4,2054.0635,6573.0030,75686.700,20140501,20140513,20140508


In [184]:
reseller_sales['businessentityid']

0       279
1       279
2       278
3       282
4       278
       ... 
6316    275
6317    279
6318    279
6319    276
6320    281
Name: businessentityid, Length: 6321, dtype: int64